<a href="https://colab.research.google.com/github/gtbnhyujmj/Gold_Cross-Exchange-Arbitrage/blob/main/%5BV1%5D%20XAUT_BX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BX 查詢價格

In [274]:
import time
import requests
import hmac
from hashlib import sha256

In [275]:
# API 主網址
APIURL = "https://open-api.bingx.com"

In [276]:
# 你的 API 金鑰（可以先空著測試）
APIKEY = ""
SECRETKEY = ""

In [277]:
# 查詢 XAUT-USDT 的即時價格 (泰達公司出的泰達黃金)

def demo():
    # 請求資料可以是空的（GET 請求）
    payload = {}

    # API 的路徑
    path = '/openApi/swap/v1/ticker/price'

    # 請求方式
    method = "GET"

    # 要送出的參數
    paramsMap = {
        "symbol": "XAUT-USDT",  # 想查詢的幣種
        "timestamp": "1702718923479"  # 固定的測試值，後面會自動更新
    }

    # 將參數轉成 URL 格式字串
    paramsStr = parseParam(paramsMap)

    # 送出請求
    return send_request(method, path, paramsStr, payload)

In [278]:
# 對參數進行簽名（加密）

def get_sign(api_secret, payload):
    # 使用 SHA256 加密並回傳簽名值
    key_bytes = api_secret.encode("utf-8")
    payload_bytes = payload.encode("utf-8")
    signature = hmac.new(key_bytes, payload_bytes, digestmod=sha256).hexdigest()
    print("簽名結果 =", signature)
    return signature

In [279]:
# 發送 GET 請求

def send_request(method, path, urlpa, payload):
    # 手動組合 URL 字串（不使用 f-string）
    base_url = APIURL + path + "?" + urlpa + "&signature=" + get_sign(SECRETKEY, urlpa)
    print("請求網址 =", base_url)

    # 設定 headers，帶上 API 金鑰
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 送出請求
    response = requests.request(method, base_url, headers=headers, data=payload)

    # 回傳伺服器的回應
    return response.text

In [280]:
# 將參數轉為 URL 字串，並自動加入目前時間戳

def parseParam(paramsMap):
    # 將參數依照 key 排序
    sortedKeys = sorted(paramsMap)

    # 將每組 key=value 用 & 串接起來
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr = paramsStr + "&" + key + "=" + str(paramsMap[key])
        else:
            paramsStr = key + "=" + str(paramsMap[key])

    # 加上目前時間戳
    now_timestamp = "timestamp=" + str(int(time.time() * 1000))

    # 如果已經有參數，就補上 &timestamp
    if paramsStr != "":
        return paramsStr + "&" + now_timestamp
    else:
        return now_timestamp

In [281]:
# 主程式：呼叫 demo 函式
if __name__ == '__main__':
    print("執行結果:", demo())

簽名結果 = aed399f3ac2398185aad63d4ff280fbe314cc6cf85414bf7d2bd816e0a4e5b98
請求網址 = https://open-api.bingx.com/openApi/swap/v1/ticker/price?symbol=XAUT-USDT&timestamp=1702718923479&timestamp=1749910464341&signature=aed399f3ac2398185aad63d4ff280fbe314cc6cf85414bf7d2bd816e0a4e5b98
執行結果: {"code":0,"msg":"","data":{"symbol":"XAUT-USDT","price":"3435.5","time":1749910464407}}


In [282]:
# 自動排版
import json  # 加在檔案開頭

BX = demo()

# 在 print(result) 改成這樣：
parsed = json.loads(BX)
print(json.dumps(parsed, indent=2))

簽名結果 = 1c20e7d7aa5e106349cf6945fb6ab8929e236e1411a2c32b6a5d4ce1dc3b3b9a
請求網址 = https://open-api.bingx.com/openApi/swap/v1/ticker/price?symbol=XAUT-USDT&timestamp=1702718923479&timestamp=1749910464679&signature=1c20e7d7aa5e106349cf6945fb6ab8929e236e1411a2c32b6a5d4ce1dc3b3b9a
{
  "code": 0,
  "msg": "",
  "data": {
    "symbol": "XAUT-USDT",
    "price": "3435.6",
    "time": 1749910464808
  }
}


In [283]:
bingx_gold_price = parsed["data"]["price"]

bingx_gold_price

'3435.6'

# 交易所2 查詢價格

In [284]:
exchange_b_gold_price = parsed["data"]["price"]

exchange_b_gold_price

'3435.6'

# BX 開倉定義

In [285]:
import time
import requests
import hmac
from hashlib import sha256

In [286]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [287]:
def BX_place_order(symbol, side, positionSide, quantity, take_profit_price=None):

    """
    向 BingX 發送市價單下單指令，可選擇停利參數。

    symbol: 幣種（例如 "BTC-USDT"）
    side: "BUY" 或 "SELL"
    positionSide: "LONG" 或 "SHORT"
    quantity: 下單數量
    take_profit_price: 停利價格（選填）

    回傳值為 BingX API 的回應文字。
    """

    # API 資訊
    path = '/openApi/swap/v2/trade/order'
    method = "POST"

    # 組裝參數
    paramsMap = {
        "symbol": symbol,
        "side": side,
        "positionSide": positionSide,
        "type": "MARKET",  # 市價單
        "quantity": quantity
    }

    # 如有設定停利價格，就加入 takeProfit 條件
    if take_profit_price is not None:
        take_profit_dict = {
            "type": "TAKE_PROFIT_MARKET",
            "stopPrice": take_profit_price,
            "price": take_profit_price,
            "workingType": "MARK_PRICE"
        }
        import json
        paramsMap["takeProfit"] = json.dumps(take_profit_dict)

    # 加入時間戳並排序
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (k, paramsMap[k]) for k in sortedKeys])
    timestamp = str(int(time.time() * 1000))
    if paramsStr != "":
        paramsStr += "&timestamp=" + timestamp
    else:
        paramsStr = "timestamp=" + timestamp

    # 產生簽名
    sign_payload = paramsStr
    signature = hmac.new(SECRETKEY.encode("utf-8"), sign_payload.encode("utf-8"), digestmod=sha256).hexdigest()

    # 發送請求
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    response = requests.request(method, url, headers=headers, data={})
    return response.text

In [288]:
# 下單 BTC-USDT，多單，數量 5，不設停利
# result = BX_place_order("BTC-USDT", "BUY", "LONG", 5)

# 下單 ETH-USDT，空單，數量 2，停利設在 3500
# result = BX_place_order("ETH-USDT", "SELL", "SHORT", 2, take_profit_price=3500)

# print("回傳結果:", result)

In [289]:
# 以BX為基準交易所，查完價格後就直接開倉。
# 因為是兩個或是未來多個交易所套利，所以應該是不用設定停利條件。

# BINGX關倉需要合約號，所以還要再寫個查合約號，並且記錄合約號。

# BX 合約號紀錄

In [290]:
import time
import requests
import hmac
from hashlib import sha256

In [291]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [292]:
def get_bingx_position_id(symbol):

    """
    查詢指定幣種持倉資訊，回傳該倉位的 positionId 字串。
    symbol: 幣種對（例如 "BNB-USDT"）

    回傳: positionId（字串），若查無則回傳 None。
    """

    # API 資訊
    path = '/openApi/swap/v2/user/positions'
    method = "GET"

    # 組裝請求參數
    paramsMap = {
        "symbol": symbol
    }

    # 排序參數並加入 timestamp
    sortedKeys = sorted(paramsMap)
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr += "&"
        paramsStr += key + "=" + str(paramsMap[key])
    paramsStr += "&timestamp=" + str(int(time.time() * 1000))

    # 簽名
    signature = hmac.new(
        SECRETKEY.encode("utf-8"),
        paramsStr.encode("utf-8"),
        digestmod=sha256
    ).hexdigest()

    # URL 組合
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    print("查詢請求：", url)

    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 發送請求
    response = requests.request(method, url, headers=headers, data={})
    try:
        result = response.json()
        position_id = result["data"]["positionId"]
        print("🎯 查到 positionId =", position_id)
        return str(position_id)
    except Exception as e:
        print("⚠️ 解析失敗或查無倉位：", e)
        return None

In [293]:
position_id = get_bingx_position_id("XAUT-USDT")

if position_id is not None:
    print("✅ 倉位 ID 已獲得：", position_id)
else:
    print("❌ 查無倉位")

查詢請求： https://open-api.bingx.com/openApi/swap/v2/user/positions?symbol=XAUT-USDT&timestamp=1749910465071&signature=3e04bed9fd608c2343f6fa24b4d5fe3a30322d0d7a147f7f048ac063c162162e
⚠️ 解析失敗或查無倉位： 'data'
❌ 查無倉位


# 交易所2 開倉定義

# BX 關倉定義

In [294]:
import time
import requests
import hmac
from hashlib import sha256

In [295]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [296]:
def close_bingx_position(order_id, symbol):

    """
    取消 BingX 的合約訂單（平倉用）

    order_id: 要取消的訂單 ID
    symbol: 幣種對，如 "RNDR-USDT"

    回傳值為 API 的回應文字。
    """

    # API 路徑與方法
    path = '/openApi/swap/v2/trade/order'
    method = "DELETE"

    # 組合參數（不含簽名）
    paramsMap = {
        "orderId": order_id,
        "symbol": symbol
    }

    # 加入 timestamp 並轉為參數字串
    sortedKeys = sorted(paramsMap)
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr = paramsStr + "&" + key + "=" + str(paramsMap[key])
        else:
            paramsStr = key + "=" + str(paramsMap[key])

    # 加入當前時間戳
    timestamp = str(int(time.time() * 1000))
    if paramsStr != "":
        paramsStr += "&timestamp=" + timestamp
    else:
        paramsStr = "timestamp=" + timestamp

    # 簽名處理
    signature = hmac.new(
        SECRETKEY.encode("utf-8"),
        paramsStr.encode("utf-8"),
        digestmod=sha256
    ).hexdigest()

    # 組合 URL
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    print("取消訂單請求：", url)

    # 設定標頭與送出請求
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    response = requests.request(method, url, headers=headers, data={})
    return response.text

# 交易所2 關倉定義

# 比價

In [297]:
import time

# 每幾秒檢查一次
CHECK_INTERVAL = 0.2  # 秒

# 當價差超過這個值就提醒
TARGET_SPREAD = 5  # 假設為 5 USDT

while True:
    bingx_price = float(bingx_gold_price)     # 交易所A（BingX）的報價
    exchb_price = float(exchange_b_gold_price)  # 交易所B的報價（尚未實作）

    # 兩邊都查到才繼續
    if bingx_price is not None and exchb_price is not None:
        # 價差 = BX - B
        spread = bingx_price - exchb_price

        print("BingX:", bingx_price, "| 交易所B:", exchb_price, "| 價差:", spread)

        # 如果價差大於目標門檻
        print("🚀 價差超過門檻！執行雙方平倉策略...")

        # 發送平倉指令（實際情況請串接交易 API）
        # close_bingx_position()
        # close_exchange_b_position()

        print("✅ 已向兩家交易所發出平倉指令")
        break  # 測試完後退出迴圈

    time.sleep(CHECK_INTERVAL)

BingX: 3435.6 | 交易所B: 3435.6 | 價差: 0.0
🚀 價差超過門檻！執行雙方平倉策略...
✅ 已向兩家交易所發出平倉指令


In [ ]:
# 我們討論的 應該是 BX - B = 0 的時候，同步平倉。
# 應該是TARGET_SPREAD = 0 的時候同步平倉，這是我的理解。

In [ ]:
# 2025/06/14